## **Supervised ML regression algorithm to predict next round team value (CT & T)**
## **Algorithm**

Input data: Obtained DataFrames from the previous prediction:
- ct_predicted_value
- t_predicted_value

Preprocces the data to create a new traget column, next round team value (**nxt_rnd_ct_val** & **nxt_rnd_t_val**)

In [1]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso

from lightgbm import LGBMRegressor
from joblib import dump, load

In [2]:
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 30)

### Load Data

In [3]:
ct_df = pd.read_csv('../data/processed/3_TEST_base_predict_next_rnd_ct_val.csv')
# t_df = pd.read_csv('../data/processed/3_base_predict_next_rnd_t_val.csv')

In [4]:
display(ct_df.head())
# display(t_df.head())

,file,round,wp_ct_val,nade_ct_val,ct_alive,t_alive,ct_winner,bomb_planted,ct_cons_wins,t_cons_wins,ct_val_pred,nxt_rnd_ct_val
0,0,1,1000.0,550,5,5,0.5,0.5,0,0,4078.134589,18450
1,0,2,10100.0,1100,4,0,1.0,0.0,1,0,17819.702711,9550
2,0,3,4125.0,900,0,1,0.0,0.0,0,1,7038.468589,1600
3,0,4,1000.0,0,0,3,0.0,1.0,0,2,1452.468928,23350
4,0,5,15500.0,1400,0,4,0.0,1.0,0,3,22676.205763,26400


### Label enconder

In [5]:
# # LabelEncoder to categorical feature -> file
# le = LabelEncoder()

# ct_df['file'] = le.fit_transform(ct_df.file)

In [6]:
# display(ct_df.tail())

In [7]:
CT_FEATS = ['file', 'round', 'wp_ct_val', 'nade_ct_val', 'ct_alive', 't_alive', 'ct_winner', 'bomb_planted', 'ct_cons_wins', 't_cons_wins', 'ct_val_pred']
# CT_SCALER_FEATS = ['file', 'round', 'wp_ct_val', 'nade_ct_val', 'ct_alive', 't_alive', 'ct_cons_wins', 't_cons_wins', 'ct_val_real']

In [8]:
scaler = StandardScaler()

# ct_df['file'] = se.fit_transform(ct_df.file)
# ct_df['round'] = se.fit_transform(ct_df.round)
# ct_df['wp_ct_val'] = se.fit_transform(ct_df.wp_ct_val)
# ct_df['nade_ct_val'] = se.fit_transform(ct_df.nade_ct_val)
# ct_df['ct_alive'] = se.fit_transform(ct_df.ct_alive)
# ct_df['t_alive'] = se.fit_transform(ct_df.t_alive)
# ct_df['ct_cons_wins'] = se.fit_transform(ct_df.ct_cons_wins)
# ct_df['t_cons_wins'] = se.fit_transform(ct_df.t_cons_wins)
# ct_df['ct_val_real'] = se.fit_transform(ct_df.ct_val_real)

ct_df[CT_FEATS] = scaler.fit_transform(ct_df[CT_FEATS])

# scaled_data = scaler.fit_transform(data)

### Define Features and Target

In [9]:
# CT_FEATS = ['file', 'round', 'wp_ct_val', 'nade_ct_val', 'ct_alive', 't_alive', 'ct_winner', 'bomb_planted', 'ct_cons_wins', 't_cons_wins', 'ct_val_real', 'ct_val_pred']
# T_FEATS = ['file', 'round', 'wp_t_val', 'nade_t_val', 'ct_alive', 't_alive', 'ct_winner', 'bomb_planted', 'ct_cons_wins', 't_cons_wins', 't_val_real', 't_val_pred']

CT_FEATS = ['file', 'round', 'wp_ct_val', 'nade_ct_val', 'ct_alive', 't_alive', 'ct_winner', 'bomb_planted', 'ct_cons_wins', 't_cons_wins', 'ct_val_pred']
# T_FEATS = ['file', 'round', 'wp_t_val', 'nade_t_val', 'ct_alive', 't_alive', 'ct_winner', 'bomb_planted', 'ct_cons_wins', 't_cons_wins', 't_val_pred']

# CT_FEATS = ['file', 'round', 'wp_ct_val', 'nade_ct_val', 'ct_alive', 't_alive', 'ct_winner', 'bomb_planted', 'ct_cons_wins', 't_cons_wins', 'ct_val_real']
# T_FEATS = ['file', 'round', 'wp_t_val', 'nade_t_val', 'ct_alive', 't_alive', 'ct_winner', 'bomb_planted', 'ct_cons_wins', 't_cons_wins', 't_val_real']

CT_TARGET = 'nxt_rnd_ct_val'
# T_TARGET = 'nxt_rnd_t_val'

### Preprocessor

In [10]:
#Preprocessor

# numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

# preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, CT_FEATS)])
# preprocessor_t = ColumnTransformer(transformers=[('num', numeric_transformer, T_FEATS)])

### **SPLITS**

In [11]:
ct_train, ct_test = train_test_split(ct_df, train_size=0.95)
# t_train, t_test = train_test_split(t_df)

In [12]:
print(ct_train.shape, ct_test.shape)
# print(t_train.shape, t_test.shape)

(295272, 12) (15541, 12)


### **MODEL**

#### We take **LightGBM** as the regressor algorithm, like the previous prediction

In [13]:
# regressor = RandomForestRegressor()

regressor = LGBMRegressor(boosting_type='gbdt', 
                       bagging_freq=1, 
                       bagging_fraction = 0.9, 
                       n_estimators=100)

# regressor = Lasso()

In [14]:
model = regressor

# t_model = Pipeline(steps=[('t_preprocessor', preprocessor_t),
#                            ('regressor', regressor)])

In [15]:
model.fit(ct_train[CT_FEATS], ct_train[CT_TARGET]);

# t_model.fit(t_train[T_FEATS], t_train[T_TARGET]);

### **CHECK PERFORMANCE**

In [16]:
y_ct_test = model.predict(ct_test[CT_FEATS])
y_ct_train = model.predict(ct_train[CT_FEATS])

# y_t_test = t_model.predict(t_test[T_FEATS])
# y_t_train = t_model.predict(t_train[T_FEATS])

In [17]:
# Mean_squared_error

print(f"CT test error: {mean_squared_error(y_pred=y_ct_test, y_true=ct_test[CT_TARGET], squared=False)}")
print(f"CT train error: {mean_squared_error(y_pred=y_ct_train, y_true=ct_train[CT_TARGET], squared=False)}")
# print()
# print(f"T test error: {mean_squared_error(y_pred=y_t_test, y_true=t_test[T_TARGET], squared=False)}")
# print(f"T train error: {mean_squared_error(y_pred=y_t_train, y_true=t_train[T_TARGET], squared=False)}")

CT test error: 8828.876747858485
CT train error: 8695.3185626894


In [18]:
# r2_score

print(f"CT test error: {r2_score(y_pred=y_ct_test, y_true=ct_test[CT_TARGET])}")
print(f"CT train error: {r2_score(y_pred=y_ct_train, y_true=ct_train[CT_TARGET])}")
print()
# print(f"T test error: {r2_score(y_pred=y_t_test, y_true=t_test[T_TARGET])}")
# print(f"T train error: {r2_score(y_pred=y_t_train, y_true=t_train[T_TARGET])}")

CT test error: 0.3370541957088644
CT train error: 0.35108500209330396



--------------------------------
### LGBM REGRESSOR
CT test error:______Base: 1402.436|| 0.82444______<font color='green'>Hyp. tuning:  || </font>

CT train error:______Base: 1131.833|| 0.88526______<font color='green'>Hyp. tuning:  || </font>

T test error:______Base: 2725.320 || 0.90388______<font color='green'>Hyp. tuning:  || </font>

T train error:______Base: 2728.544 || 0.90519______<font color='green'>Hyp. tuning:  || </font>

In [25]:
ct_df['nxt_rnd_ct_val']

0         18450
1          9550
2          1600
3         23350
4         26400
          ...  
310808    30950
310809    32900
310810    25950
310811    14300
310812        0
Name: nxt_rnd_ct_val, Length: 310813, dtype: int64

In [26]:
ct_test['nxt_rnd_ct_val']

14723      9400
28382     24700
150339    18650
170529    19700
119580     7750
          ...  
33759     22550
250924    28750
269372    25550
99375     15700
115951    27000
Name: nxt_rnd_ct_val, Length: 15541, dtype: int64

In [20]:
y_ct_test

array([11612.20214493, 22903.44308688, 11650.43326715, ...,
        8896.51611329, 18874.14255767, 21995.0721425 ])

In [19]:
ct_df.describe()

,file,round,wp_ct_val,nade_ct_val,ct_alive,t_alive,ct_winner,bomb_planted,ct_cons_wins,t_cons_wins,ct_val_pred,nxt_rnd_ct_val
count,3.108130e+05,3.108130e+05,3.108130e+05,3.108130e+05,3.108130e+05,3.108130e+05,3.108130e+05,3.108130e+05,3.108130e+05,3.108130e+05,3.108130e+05,310813.000000
mean,-2.340944e-17,-7.384032e-17,-4.082935e-17,-3.612003e-17,-1.053882e-17,-7.315449e-17,1.146697e-16,-6.785079e-17,6.565616e-17,1.108748e-16,-1.212993e-16,18728.716302
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,10796.743553
min,-1.731171e+00,-1.572378e+00,-1.727419e+00,-1.315128e+00,-3.866275e+00,-3.844742e+00,-9.990962e-01,-9.634300e-01,-6.538585e-01,-6.816671e-01,-1.771653e+00,0.000000
25%,-8.658370e-01,-8.257729e-01,-1.053057e+00,-8.999871e-01,-9.446750e-01,-8.989763e-01,-9.990962e-01,-9.634300e-01,-6.538585e-01,-6.816671e-01,-1.146586e+00,7400.000000
50%,-2.191404e-04,-7.916738e-02,2.559981e-01,-1.388954e-01,-3.603550e-01,-3.098232e-01,2.133358e-02,-9.634300e-01,-6.538585e-01,-6.816671e-01,3.169390e-01,21750.000000
75%,8.651144e-01,7.918724e-01,7.816038e-01,6.913864e-01,8.082848e-01,8.684830e-01,1.041763e+00,1.080455e+00,4.132478e-01,3.768834e-01,8.849886e-01,28150.000000
max,1.733576e+00,5.769242e+00,2.358421e+00,5.119556e+00,1.976925e+00,2.046789e+00,1.041763e+00,1.080455e+00,1.055076e+01,1.149166e+01,1.976323e+00,42050.000000
